# Project part 2

Aaron

In [2]:
# Imports
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
import random
import seaborn as sns
import scipy.stats as stats
import uuid
from bson import ObjectId
from cassandra.cluster import Cluster
from datetime import datetime
from io import StringIO
from geopy import Nominatim
from meteostat import Hourly, Stations, Point
from pymongoarrow.api import write
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from scipy.fft import dct, idct

# Use a ggplot theme when plotting
plt.style.use("ggplot")

In [3]:
# Connecting to MongoDB
url = (
    "mongodb+srv://medvetslos:"
    + json.load(open("../../.nosync/mongoDB.json"))["pwd"]
    + "@ind320-project.lunku.mongodb.net/?retryWrites=true&w=majority&appName=IND320-project"
)

mdb_client = MongoClient(url, server_api=ServerApi("1"))

try:
    mdb_client.admin.command("ping")
    print("Pinged your deployment. Successfully connected to MongoDB.")
except Exception as exceptionMsg:
    print(exceptionMsg)

database = mdb_client["IND320-project"]  # Retrieving MongoDB collections
municipalities = database[
    "municipalities"
]  # Access the "muncipalities" collection

Pinged your deployment. Successfully connected to MongoDB.


In [4]:
# Set environment variables to make PySpark work
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/"
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"
os.environ["PYSPARK_HADOOP_VERSION"] = "without"

# Spark set up
spark = (
    SparkSession.builder.appName("SparkCassandraApp")
    .config(
        "spark.jars.packages",
        "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1",
    )
    .config("spark.cassandra.connection.host", "localhost")
    .config(
        "spark.sql.extensions",
        "com.datastax.spark.connector.CassandraSparkExtensions",
    )
    .config(
        "spark.sql.catalog.mycatalog",
        "com.datastax.spark.connector.datasource.CassandraCatalog",
    )
    .config("spark.cassandra.connection.port", "9042")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g")
    .config("spark.task.maxFailures", "10")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

keyspace = "ind320_project"

24/11/13 12:55:22 WARN Utils: Your hostname, Aarons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.11.132 instead (on interface en0)
24/11/13 12:55:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/aaron/.ivy2/cache
The jars for the packages stored in: /Users/aaron/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d6e48d7-8c50-43d1-924b-fc7db049f721;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central


:: loading settings :: url = jar:file:/Users/aaron/Documents/IND320_projects/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in central
	found com.datastax.oss#java-driver-query-builder;4.13.0 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.11 in central
:: resolution report :: resolve 200ms :: artifacts dl 7ms
	:: modules in use:
	com.datastax.oss#java-d

In [5]:
cluster = Cluster(["localhost"], port=9042)
session = cluster.connect()
keyspace = "ind320_project"

# Creating a keyspace in Cassandra
session.execute(
    "CREATE KEYSPACE IF NOT EXISTS"
    + " "
    + keyspace
    + " "
    + "WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1};"
)

session.set_keyspace(
    keyspace
)  # Setting the keyspace to be able to retrieve my tables

In [6]:
# Functions from assignment 1:

# Creating a view to Cassandra with PySpark
def create_cassandra_view(view_name: str, table_name: str, keyspace_name: str):
    spark.read.format("org.apache.spark.sql.cassandra").options(
        table=table_name, keyspace=keyspace_name
    ).load().createOrReplaceTempView(view_name)
    print(f"View '{view_name}' created.")


# Creating a create table query for Cassandra.
def create_cassandra_table_query(df, new_table_name, keyspace_name):
    # Define mapping between Pandas and Cassandra datatypes
    dtype_mapping = {
        "int64": "int",
        "float64": "double",
        "object": "text",
        "bool": "boolean",
        "datetime64[ns]": "timestamp",
    }

    # Start constructing the CREATE TABLE query
    query = f"CREATE TABLE IF NOT EXISTS {keyspace_name}.{new_table_name} (\n"

    # Add primary key column with timeuuid
    columns = ["id timeuuid"]

    # Add remaining columns with mapped Cassandra data types
    for col, dtype in df.dtypes.items():
        if col != "id":  # Exclude 'id' to avoid duplication
            cassandra_type = dtype_mapping.get(
                str(dtype), "text"
            )  # Default to 'text' if type is unrecognized
            columns.append(f"{col} {cassandra_type}")

    # Join columns with commas and specify primary key as 'id'
    columns_str = ",\n    ".join(columns)
    query += f"    {columns_str},\n"
    query += "    PRIMARY KEY (id)\n);"

    return query


# Inserting a Pandas DataFrame into Cassandra with PySpark
def insert_to_cassandra(df: pd.DataFrame, table_name: str, keyspace_name: str):
    # Prompt: How can I add a timeuuid 'id' column when inserting into Cassandra with PySpark
    def generate_timeuuid():
        return str(uuid.uuid1())

    timeuuid_udf = udf(generate_timeuuid, StringType())
    spark.createDataFrame(df).withColumn("id", timeuuid_udf()).write.format(
        "org.apache.spark.sql.cassandra"
    ).mode("append").options(table=table_name, keyspace=keyspace_name).save()

    print("Insertion completed!")

### Geographical locations of municipalities

In [ ]:
# Define price_areas based off of energidataservice.dk's definition:
# "DK1 is west of the Great Belt and DK2 is east of the Great Belt."
# https://www.energidataservice.dk/tso-electricity/productionconsumptionsettlement

price_areas = {
    "DK1": ["North", "Central", "South"],
    "DK2": ["Zealand", "Capital"],
}

dk2 = ["Zealand", "Capital"]

nominatim = Nominatim(
    user_agent="personal-application, project"
)  # Initialize a geocoder from GeoPy

# Retrieve only "Region" and "Municipality" from the MongoDB collection
region_municipality = municipalities.find({}, {"Region": 1, "Municipality": 1})

# Iterate over every municipality
for row in region_municipality:
    _id = row["_id"]

    # # Set price area
    # if row["Region"] in price_areas["DK2"]:
    #     price_area = "DK2"
    # else:
    #     price_area = "DK1"

    # Set price area
    if row["Region"] in dk2:
        price_area = "DK2"
    else:
        price_area = "DK1"

    # Find longitude and latitude of the municipality
    geocode_location = nominatim.geocode(row["Municipality"] + 
                                         ", Denmark")
    longitude = geocode_location.longitude
    latitude = geocode_location.latitude

    # Update the MongoDB entry with new values
    update_template = {
        "$set": {
            "Price Area": price_area,
            "Longitude": longitude,
            "Latitude": latitude,
        }
    }
    municipalities.update_one({"_id": _id}, update_template)

### Weather data

In [10]:
# Defining time period we want data from

# Update 24-11-13: Since MeteoStat uses UTC we change the start and end 
# accordingly such that it aligns with our energy data.

start = datetime(2021, 12, 31, 23, 00)
end = datetime(2022, 12, 31, 22, 00)

# Initialize a MeteoStat station object
stations = Stations()

df_weather = pd.DataFrame()

municipality_coords = municipalities.find(
    {}, {"Municipality": 1, "Longitude": 1, "Latitude": 1}
)

for row in municipality_coords:
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    # print(row[
    #     "Municipality"
    # ])
    nearby_stations = stations.nearby(latitude, longitude)
    nearest_station = nearby_stations.fetch(
        1
    )  # Fetches nearest station to the coordinates given
    distance = nearest_station["distance"].iloc[
        0
    ]  # Returns distance to nearest station

    weather_location = Hourly(
        Point(latitude, longitude), start, end
    )  # Fetches hourly weather data for the nearest station during the defined period
    weather_data = (
        weather_location.fetch()
    )  # Retrieves hourly weather data for specified period
    weather_data["distance_to_station"] = (
        distance  # Creating a column for the distance
    )
    weather_data["municipality"] = row[
        "Municipality"
    ]  # Adding a "municipality" column so we can distinguish the data

    df_weather = pd.concat(
        [df_weather, weather_data]
    )  # Concatenating the weather data subsequently

df_weather = df_weather.reset_index()
df_weather = df_weather.rename(columns={"index": "hourutc"})

In [ ]:
df_weather.shape
# df_weather[df_weather["municipality"] == "Aarhus"]

In [11]:
df_weather

,hourutc,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,distance_to_station,municipality
0,2021-12-31 23:00:00,7.5,6.9,96.0,0.0,NaN,284.0,16.2,34.6,1011.9,NaN,5.0,2336.324166,Copenhagen
1,2022-01-01 00:00:00,7.5,7.2,98.0,0.0,NaN,275.0,16.9,33.1,1012.4,NaN,5.0,2336.324166,Copenhagen
2,2022-01-01 01:00:00,7.4,7.3,99.0,0.0,NaN,277.0,16.2,32.8,1013.1,NaN,5.0,2336.324166,Copenhagen
3,2022-01-01 02:00:00,7.3,7.2,99.0,0.0,NaN,285.0,15.8,31.3,1013.5,NaN,5.0,2336.324166,Copenhagen
4,2022-01-01 03:00:00,7.2,6.9,98.0,0.0,NaN,290.0,18.4,36.0,1013.9,NaN,5.0,2336.324166,Copenhagen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744595,2022-12-31 18:00:00,7.7,7.6,99.0,0.0,NaN,210.0,7.6,20.4,999.9,NaN,8.0,16705.854121,Fanø
744596,2022-12-31 19:00:00,8.0,8.0,100.0,1.2,NaN,180.0,7.6,20.4,1000.0,NaN,8.0,16705.854121,Fanø
744597,2022-12-31 20:00:00,7.0,7.0,100.0,1.0,NaN,170.0,9.4,18.5,999.0,NaN,9.0,16705.854121,Fanø
744598,2022-12-31 21:00:00,7.0,6.9,99.0,0.0,NaN,120.0,5.4,22.2,999.6,NaN,9.0,16705.854121,Fanø


In [12]:
cas_weather_table = "weather_data"  # Defining a name for my Cassandra table
session.execute(
    f"DROP TABLE IF EXISTS {keyspace}.{cas_weather_table}"
)  # Executing DROP TABLE
weather_table = create_cassandra_table_query(
    df_weather, cas_weather_table, keyspace
)  # Creating a CREATE TABLE query based off of the DataFrame df_weather
session.execute(weather_table)
insert_to_cassandra(df_weather, cas_weather_table, keyspace)

24/11/13 12:57:51 WARN TaskSetManager: Stage 0 contains a task of very large size (8072 KiB). The maximum recommended task size is 1000 KiB.


Insertion completed!


## Smoothing and outliers/anomalies

### Gas prices

In [ ]:
# Get gas prices from the MongoDB collection "gas"
gas_prices = [
    entry
    for entry in database["gas"].find(
        {}, {"PurchasePriceDKK_kWh": 1, "GasDay": 1}
    )
]

df_gas_prices = pd.DataFrame.from_records(
    gas_prices
)  # Make a DataFrame from the list of gas prices
df_gas_prices = df_gas_prices.sort_values(
    "GasDay", ascending=True
)  # Sort the dataframe in temporal order
df_gas_prices.head()

In [ ]:
W = np.arange(0, df_gas_prices.shape[0])
dct_gas_prices = dct(
    df_gas_prices["PurchasePriceDKK_kWh"].values
)  # DCT on the gas prices

# Plotting of the DCT
num_coeff = 15

plt.figure(figsize=(10, 5))
plt.plot(np.abs(dct_gas_prices))
plt.xlabel("coefficient")
plt.ylabel("amplitude")
# plt.xlim(0, dct_gas_prices.shape[0])
plt.xlim(0, num_coeff)
plt.xticks(np.arange(0, num_coeff + 1, step=2))
plt.show()

In [ ]:
dct_gas_prices[(W > 3)] = 0  # Filter out frequencies greater than 3

plt.figure(figsize=(10, 5))
plt.fill_between(
    df_gas_prices["GasDay"],
    df_gas_prices["PurchasePriceDKK_kWh"],
    color="grey",
    alpha=0.5,
    label="Original",
)
plt.xlabel("Date")
plt.ylabel("Gas Price")
plt.title("Gas Prices Over Time")
plt.plot(
    df_gas_prices["GasDay"],
    idct(dct_gas_prices),
    label="Filtered",
    color="purple",
    linewidth=6,
)
plt.legend()
plt.show()

### Electricity exchange

In [ ]:
create_cassandra_view(
    "prodcons_view", "prodcons", keyspace
)  # Create a view to access the "prodcons" table
df_exchangeNO = spark.sql(
    "SELECT hourdk, exchangeno_mwh FROM prodcons_view"
).toPandas()
df_exchangeNO = df_exchangeNO.groupby("hourdk").aggregate(
    {"exchangeno_mwh": "sum"}
)  # Aggregate sum such that we get the total exchange for DK1 and DK2

# Mean and standard deviation of the electricity exchange
mean_exchangeNO = df_exchangeNO["exchangeno_mwh"].mean()
std_exchangeNO = df_exchangeNO["exchangeno_mwh"].std()
print(f"Mean: {mean_exchangeNO:.1f}")
print(f"Standard deviation: {std_exchangeNO:.1f}")

In 2022, the danes on average imported $228.6\ MWh$ of energy a day from Norway with a standard deviation of $1013\ MWh$. The standard deviation is big as it constantly jumps between importing and exporting energy. 

In [ ]:
# Robust statistics
k = 1.4826  # k with the assumption that our data is normally distributed
trim_mean_exchangeNO = stats.trim_mean(
    df_exchangeNO["exchangeno_mwh"], 0.05
)  # Trimmed mean removing 5% of the most extreme observations
MAD_exchangeNO = stats.median_abs_deviation(df_exchangeNO["exchangeno_mwh"])
print(f"Trimmed mean: {trim_mean_exchangeNO:.3f}")
print(f"MAD: {MAD_exchangeNO:.3f}")
print(f"Adjusted MAD: {MAD_exchangeNO * k:.3f}")


Trimmed mean is different from the normal mean statistic as we remove a percentage of the most extreme observations, here 5%. Since we are removing a proportion, the number of samples becomes smaller which results in a "bigger" mean value than the original mean.

Median absolute devation differs from standard deviation, but it has a relation to it by multiplying it by $k \approx 1.4826$ if we assume that our data is normally distributed. By multiplying by $k$ we get a value more similair to the standard deviation $\hat{\sigma}$ approximated above. We see that our adjusted $MAD$-value differs slightly from the standard deviation we estimated above.

In [ ]:
# Plotting the gas prices with standard deviations and mean
plt.figure(figsize=(10, 5))
plt.plot(df_exchangeNO.index, df_exchangeNO["exchangeno_mwh"], color="blue")
plt.title("Exchange of energy Denmark-Norway 2022")
plt.axhline(mean_exchangeNO, color="black", label="Mean")
plt.axhline(mean_exchangeNO - std_exchangeNO, color="red", label="-1 Std Dev")
plt.axhline(mean_exchangeNO + std_exchangeNO, color="red", label="+1 Std Dev")
plt.axhline(
    mean_exchangeNO - 2 * std_exchangeNO,
    color="red",
    linestyle="--",
    label="-2 Std Dev",
)
plt.axhline(
    mean_exchangeNO + 2 * std_exchangeNO,
    color="red",
    linestyle="--",
    label="+2 Std Dev",
)
plt.legend()
plt.show()

In [ ]:
# Calculating +- 3 Standard deviations
upper_spc = mean_exchangeNO + 3 * std_exchangeNO
lower_spc = mean_exchangeNO - 3 * std_exchangeNO

print(f"{lower_spc:.1f}", f"{upper_spc:.1f}")

If we assume any measurement outside $\pm 3$ to be an outlier, the export of energy has to be greater than $2810.4\ MWh$ or the import has to be greater than $3267.5\ MWh$ before we flag it.

## Imputation

In [ ]:
# Get gas prices from my MongoDB collection "gas"
gas_prices = [entry for entry in database["gas"].find()]

# Create views to access my Cassandra tables
create_cassandra_view("prodcons_view", "prodcons", keyspace)
create_cassandra_view("production_view", "production", keyspace)
create_cassandra_view("consumption_view", "consumption", keyspace)
create_cassandra_view("weather_view", "weather_data", keyspace)

# Retrieve and prepare DataFrames
df_municipalities = pd.DataFrame.from_records(
    [entry for entry in municipalities.find({})]
)
df_gas_prices = pd.DataFrame.from_records(gas_prices)
df_prodcons = spark.sql("SELECT * FROM prodcons_view").toPandas()
df_production = spark.sql("SELECT * FROM production_view").toPandas()
df_consumption = spark.sql("SELECT * FROM consumption_view").toPandas()
df_weather_data = spark.sql("SELECT * FROM weather_view").toPandas()

### Checking for missing values in each dataframe

In [ ]:
dfs = {
    "gas_prices": df_gas_prices,
    "prodcons": df_prodcons,
    "production": df_production,
    "consumption": df_consumption,
    "weather_data": df_weather_data,
    "municipalities": df_municipalities,
}

for key, df in dfs.items():
    na_df = df.isna().sum()  # Get sum of NAs for every column of the DataFrame
    non_zero_na_df = na_df[
        na_df > 0
    ]  # Filter out the columns that does not have NAs

    # Print out a summary of missing values in a DataFrame if there is any
    if not non_zero_na_df.empty:
        print(f"\n{key} - {df.shape}:")
        print(non_zero_na_df)

In [15]:
# A function to create an overview of the number and proportion missing
# values in a DataFrame
def na_overview(df: pd.DataFrame):
    na_df = df.isna().sum()
    na_df = na_df[na_df > 0].to_frame(name="# missing values")
    na_df["% missing values"] = na_df["# missing values"] / df.shape[0]
    print(df.shape)
    return na_df

#### Gas prices

In [ ]:
gas_prices = [entry for entry in database["gas"].find()]
df_gas_prices = pd.DataFrame.from_records(gas_prices)
df_gas_prices = df_gas_prices.sort_values("GasDay", ascending=True)

In [ ]:
na_gas = na_overview(df_gas_prices)
na_gas

In [ ]:
df_gas_prices.head()

In [ ]:
def plot_NA(df: pd.DataFrame, value_column: str, time_column: str):
    df = df.copy()
    df.set_index(time_column, inplace=True)

    df_not_null = df[value_column].notnull()

    plt.figure(figsize=(10, 5))
    plt.vlines(df_not_null.index, ymin=0, ymax=1, 
               colors=df_not_null.map({True: 'black', False: 'white'}), 
               linewidth=1)
    plt.xlabel('Time')
    plt.ylabel('Presence of Data')
    plt.title(f'Presence of {value_column} Over Time')
    plt.show()

In [ ]:
plot_NA(df_gas_prices, "EEXHighestPricePurchaseDKK_kWh", "GasDay")

In [ ]:
na_gas = na_overview(df_gas_prices)
na_gas

In [ ]:
    # # Rolling average
    # for col in impute_cols:
    #     temp_df[col] = temp_df[col].fillna(temp_df[col].rolling(window=365, 
    #                                                             min_periods=1).mean())

In [ ]:
from bson import ObjectId

gas_prices = [entry for entry in database["gas"].find()]
df_gas_prices = pd.DataFrame.from_records(gas_prices)
df_gas_prices = df_gas_prices.sort_values("GasDay", ascending=True)

df_gas_prices["EEXHighestPricePurchaseDKK_kWh"] = df_gas_prices[
    "EEXHighestPricePurchaseDKK_kWh"
].fillna(df_gas_prices["EEXHighestPricePurchaseDKK_kWh"].rolling(window=3, min_periods=1).mean())

df_gas_prices["EEXLowestPriceSaleDKK_kWh"] = df_gas_prices[
    "EEXLowestPriceSaleDKK_kWh"
].fillna(df_gas_prices["EEXLowestPriceSaleDKK_kWh"].rolling(window=3, min_periods=1).mean())

# df_gas_prices["_id"] = (
#     df_gas_prices["_id"].apply(lambda x: ObjectId(x)).astype(str)
# )

# write(database["gas"], df_gas_prices)

In [ ]:
na_gas = na_overview(df_gas_prices)
na_gas

#### Prodcons

In [ ]:
# Get an overview of the number and proportion of missing values
na_prodcons = na_overview(df_prodcons)
na_prodcons

In [ ]:
# Trying to assess a pattern as the missing values makes out either half
# of the dataset or its entire length
df_prodcons_na = df_prodcons[
    [
        "hourdk",
        "pricearea",
        "exchangeno_mwh",
        "exchangenl_mwh",
        "exchangegb_mwh",
    ]
].copy()
df_prodcons_na = df_prodcons_na.sort_values("hourdk", ascending=True)
df_prodcons_na.head(8)

In [ ]:
# Checking if the Netherlands and Norway only exchange energy with pricearea
# DK2 during this period. If they happen to  do so, we should see "False"
# printed out.
print(
    df_prodcons[df_prodcons["exchangeno_mwh"].notna()]
    .query("pricearea == 'DK2'")
    .empty
)
print(
    df_prodcons[df_prodcons["exchangenl_mwh"].notna()]
    .query("pricearea == 'DK2'")
    .empty
)

For **prodcons** we have missing data for the exchange of energy to Great Britain, the Netherlands and Norway. Great Britain has no import or export of energy during the period of our data, hence 17520 total missing values. For both the Netherlands and Norway we have half of the missing values as Great Britain. This can be attributed to these countries only exchanging energy with `pricearea` "DK1". We can interperet the NaNs as lack of activity during this period.

Imputing any of these missing values would not be appropriate.

In [ ]:
df_prodcons = df_prodcons.fillna(0)
na_overview

#### Production

In [ ]:
# Get an overview of the number and proportion of missing values
create_cassandra_view("production_view", "production", keyspace)
df_production = spark.sql("SELECT * FROM production_view").toPandas()
na_production = na_overview(df_production)
na_production

**production** is missing quite a lot of data in its offshore wind production, both missing more than 90% of data. The fraction of values missing would not be appropriate to impute.

However, solar and thermal energy production is missing less data which would be more feasible to impute.

In [ ]:
# Group by municipality to check misising values
na_municipality = df_production.groupby("municipalityno").apply(
    lambda x: x.isna().sum()
)

na_municipality


In [ ]:
filtered_na_municipality = na_municipality[(na_municipality > 0)].dropna(
    how="all"
)

# Number of municipalities missing measurements for the entire period
# for the different production types
for production_type in filtered_na_municipality.columns[4:]:
    print(
        f"{production_type}:",
        len(filtered_na_municipality.query(f"{production_type} == 8760")),
    )

93 and 91 municipalities are missing offshore wind production, 1 is missing solar production and 13 thermal power production during the entire period. For these municipalities, setting the missing values to 0 would be appropriate.

In [ ]:
# df_production_copy = df_production.copy()
for production_type in df_production.columns[4:]:
    temp_df = na_municipality.query(f"{production_type} == 8760")

    municipality_no = temp_df.index.tolist()

    # Prompt: How can I select the municipalities from municipality_no in
    # df_production_copy to then set all of the NaN values of the
    # production_type to 0?
    df_production.loc[
        df_production["municipalityno"].isin(municipality_no),
        production_type,
    ] = df_production.loc[
        df_production["municipalityno"].isin(municipality_no),
        production_type,
    ].fillna(0)

na_overview(df_production)

In [ ]:
# Check which municipalities the remaining missing values belong to
na_municipality_2 = df_production.groupby("municipalityno").apply(
    lambda x: x.isna().sum()
)
na_municipality_2 = na_municipality_2[na_municipality_2 > 0].dropna(how="all")
na_municipality_2

In [ ]:
print("Municipality no. 411:", df_production.query("municipalityno == 411").shape)
print("Municipality no. 151:",df_production.query("municipalityno == 151").shape)

Municipality '411' is Christiansøe which is not a municipality, but an island governed by the state through the Danish Ministry of Defence. Comparing the shape of Christiansøe against an arbitrary municipality, we see that data from Christiansøe have not been recorded over the same length of period. Here we can also replace the NaNs for the offshore wind production types with 0 as well.

In [ ]:
df_production[df_production["municipalityno"] == 411] = \
    df_production[df_production["municipalityno"] == 411].fillna(0)

na_overview(df_production.query("municipalityno == 411"))

In [ ]:
# Prompt: Please fix this code such that I can do a rolling mean over 
#         "offshorewindge100mw_mwh" for municipalityno 665 

# Filter the DataFrame for municipalityno 665
df_665 = df_production[df_production["municipalityno"] == 665].copy()

# Apply the rolling mean to the 'offshorewindge100mw_mwh' column
df_665["offshorewindge100mw_mwh"] = df_665["offshorewindge100mw_mwh"].fillna(
    df_665["offshorewindge100mw_mwh"].rolling(window=3, min_periods=1).mean()
)

# Update the original DataFrame with the imputed values
df_production.loc[df_production["municipalityno"] == 665, 
                  "offshorewindge100mw_mwh"] = df_665["offshorewindge100mw_mwh"]

na_overview(df_production.query("municipalityno == 665"))

In [ ]:
# Prompt: Please fix this code such that I can do a rolling mean over 
#         "thermalpowermwh" for municipalityno 360 

# Filter the DataFrame for municipalityno 360
df_360 = df_production[df_production["municipalityno"] == 360].copy()

# # Apply the rolling mean to the 'thermalpowermwh' column
# df_360["thermalpowermwh"] = df_360["thermalpowermwh"].fillna(
#     df_360["thermalpowermwh"].rolling(window=3, min_periods=1).mean()
# )

df_360["thermalpowermwh"] = df_360["thermalpowermwh"].interpolate("linear")

# Update the original DataFrame with the imputed values
df_production.loc[df_production["municipalityno"] == 360, 
                  "thermalpowermwh"] = df_360["thermalpowermwh"]

na_overview(df_production.query("municipalityno == 360"))

In [ ]:
# Writing imputated dataframe to Spark
from pyspark.sql.functions import col

spark.createDataFrame(df_production).withColumn("id", col("id").cast(StringType())).write \
    .format("org.apache.spark.sql.cassandra") \
    .mode('append') \
    .options(table="production", keyspace=keyspace) \
    .save()

#### Weather data (old)

In [ ]:
# # Checking the number and proportion of missing values in the weather DataFrame
# na_weather = na_overview(df_weather_data)
# na_weather

In [ ]:
# # Finding the stations where "snow" and "tsun" is being measured to assess
# # why there are many missing values
# df_snow = df_weather_data[df_weather_data["snow"].notna()]
# df_tsun = df_weather_data[df_weather_data["tsun"].notna()]
# print(df_snow["municipality"].unique())
# print(df_tsun["municipality"].unique())

In [ ]:
# # Check if there are any missing values for the locations which measures `snow` and `tsun`
# tsun_snow_locations = df_snow["municipality"].unique()
# df_tsun_snow = df_weather_data[df_weather_data["municipality"].isin(tsun_snow_locations)]
# df_tsun_snow_na = df_tsun_snow.isna().sum()
# df_tsun_snow_na[df_tsun_snow_na> 0]

**weather_data** is missing a lot of data for `snow` which at first glance is sensible since there isn't snow all year round. But upon further investigation, `snow` is only measured at a handful of stations. This is also the case for `tsun` which is measured at the same stations, hence the big number of missing values. Having to impute this for the rest of the stations would be unreasonable as no measurements exists at these locations. 

In [ ]:
# # Retrieve all municipalities with precipitation measurements
# prcp_measured = df_weather_data[df_weather_data["prcp"].notna()]["municipality"].unique()

# # Use the symmetric difference operator "^".
# # Between two sets, it returns values not shared by both sets.
# # In this case we return municipalities which does not measure precipitation at all.
# na_prcp = set(prcp_measured)^set(df_weather_data["municipality"].unique())

# print(na_prcp)

The missing values for `prcp` is partly due to 10 stations not measuring percipitation at all. Imputing the precipitation for these stations would not make sense. The remaining missing values comes from stations which do measure percipitation and these we can impute.

`dwpt`, `pres`, `rhum`, and `wspd` are missing the same amount of data.

`coco` is an integer indicating the weather condition as noted in the [documentation](https://dev.meteostat.net/formats.html#weather-condition-codes). Imputing these could give a false indication of the weather. If wanted to impute this, perhaps a LOCF would be sensible as long as the gaps are not long.

For the remaining features it would be reasonable to impute the missing values as the numbers are relatively small.

##### Imputation

In [ ]:
# # Preparing DataFrame for interpolation
# df_weather_data = df_weather_data.sort_values("datetime", ascending=True)
# df_weather_data.set_index("datetime", inplace=True)

In [ ]:
# # Impute `tsun` and `snow` for 'Sønderborg', 'Ærø', 'Lolland', 'Aabenraa' and 'Tønder'

# # Get municipalities which measures `tsun` and `snow`
# loc_snow_tsun = df_weather_data[df_weather_data["tsun"].notna()]["municipality"].unique()

# # Creating a copy with our desired municipalities
# df_snow_tsun = df_weather_data[df_weather_data["municipality"].isin(loc_snow_tsun)].copy()

# # Imputing the values by using "time" in .interpolation
# df_snow_tsun[["snow", "tsun"]] = df_snow_tsun[["snow", "tsun"]].interpolate("time")

# # Replace the existing "snow" and "tsun" measurements with their imputed counterparts
# df_weather_data.loc[df_weather_data["municipality"].isin(loc_snow_tsun),
#                     ["snow", "tsun"]] = df_snow_tsun[["snow", "tsun"]].values

In [ ]:
# # Impute `prcp` for every municipality but 'Morsø', 'Lemvig', 'Helsingør', 'Struer', 'Halsnæs', 'Odsherred', 'Skive', 'Gribskov', 'Thisted' and 'Vesthimmerland'
# exclude_list = list(na_prcp)

# # Excluding municipalities/weather stations that does not measure precipitation
# df_weather_data_prcp = df_weather_data[~df_weather_data["municipality"].isin(exclude_list)].copy()

# # Imputing the values by using "time" in .interpolation
# df_weather_data_prcp["prcp"] = df_weather_data_prcp["prcp"].interpolate("time")

# # Insert the imputed `prcp`-values where they belong
# df_weather_data.loc[~df_weather_data["municipality"].isin(exclude_list), ["prcp"]] = df_weather_data_prcp["prcp"].values

In [ ]:
# # Impute remaining missing values for columns not named `prcp`, `snow`, `tsun` and `coco`

# # Exlcude the following columns from the imputation procedure
# # since we've imputed them as seen above
# exclude_cols = ["prcp", "snow", "tsun", "coco"]

# # Create a copy and impute values
# df_weather_impute = df_weather_data.copy().drop(columns=exclude_cols)
# df_weather_impute = df_weather_impute.interpolate("time")

# # Insert the imputed values to their respective places
# df_weather_data.update(df_weather_impute)
# df_weather_data.reset_index(inplace=True)

In [ ]:
# # Double check to see if have managed to impute values
# # in our desired columns
# na_overview(df_weather_data)

#### Weather data (new)

In [63]:
create_cassandra_view("weather_view", "weather_data", keyspace)
df_weather_data = spark.sql("SELECT * FROM weather_view").toPandas()

View 'weather_view' created.


In [64]:
# Checking the number and proportion of missing values in the weather DataFrame
na_weather = na_overview(df_weather_data)
na_weather

(744600, 15)


,# missing values,% missing values
coco,48329,0.064906
dwpt,2395,0.003216
prcp,142755,0.191720
pres,2395,0.003216
rhum,2395,0.003216
snow,718330,0.964719
temp,2395,0.003216
tsun,718326,0.964714
wdir,2397,0.003219
wpgt,2477,0.003327


In [65]:
# To gauge how long a series would be for one municipality
df_weather_data.query("municipality == 'Copenhagen'").shape

(8760, 15)

In [66]:
# df_weather_data = spark.sql("SELECT * FROM weather_view").toPandas()
df_weather_data = df_weather_data.sort_values("hourutc", ascending=True)
grouped_weather = df_weather_data.groupby("municipality").apply(
    lambda x: x.isna().sum()
)
grouped_weather

,id,coco,distance_to_station,dwpt,hourutc,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
municipality,,,,,,,,,,,,,,,
Aabenraa,0,215,0,0,0,0,0,0,0,8760,0,8760,0,0,0
Aalborg,0,233,0,0,0,0,3169,0,0,8760,0,8760,0,0,0
Aarhus,0,241,0,0,0,0,3172,0,0,8760,0,8760,0,0,0
Albertslund,0,236,0,0,0,0,17,0,0,8760,0,8760,0,0,0
Allerød,0,239,0,0,0,0,17,0,0,8760,0,8760,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vejle,0,233,0,0,0,0,19,0,0,8760,0,8760,0,0,0
Vesthimmerland,0,264,0,263,0,0,8760,263,263,8760,263,8760,263,263,263
Viborg,0,232,0,0,0,0,3169,0,0,8760,0,8760,0,0,0


From a quick glance of looking at the municipalities and its missing values, many of them are a total of 8760 in various columns, making out the entire time series. For these columns and municipalities, imputing data would not be sufficient.

In addition, various municipalities are missing values for `coco` which acts as a categorical variable per Meteostat's documentation so I will refrain from imputing over this column.

In [67]:
grouped_weather[grouped_weather == 8760].dropna(how="all")

,id,coco,distance_to_station,dwpt,hourutc,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
municipality,,,,,,,,,,,,,,,
Aabenraa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
Aalborg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
Aarhus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
Albertslund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
Allerød,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vejen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
Vejle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN
Vesthimmerland,NaN,NaN,NaN,NaN,NaN,NaN,8760.0,NaN,NaN,8760.0,NaN,8760.0,NaN,NaN,NaN


In [68]:
df_weather_data_c = df_weather_data.copy()
df_weather_data_c.set_index("hourutc", inplace=True)

window_size = 24

for municipality in df_weather_data["municipality"].unique():
    # Get relevant columns to impute over
    na_weather_count = grouped_weather[grouped_weather.index == municipality]
    impute_cols = na_weather_count.loc[
        :, (na_weather_count != 0).all() & (na_weather_count < 8760).all()
    ]
    impute_cols = impute_cols.drop(
        columns=["coco"], errors="ignore"
    ).columns.tolist()

    # Impute over relevant columns
    temp_df = df_weather_data_c[
        df_weather_data_c["municipality"] == municipality
    ]

    # df_weather_data_c.loc[
    #     df_weather_data_c["municipality"] == municipality, impute_cols
    # ] = temp_df[impute_cols].interpolate("time", axis=0)

    df_weather_data_c.loc[
        df_weather_data_c["municipality"] == municipality, impute_cols
    ] = temp_df[impute_cols].interpolate("linear")


    # # Rolling average
    # for col in impute_cols:
    #     temp_df[col] = temp_df[col].fillna(temp_df[col].rolling(window=window_size, 
    #                                                             min_periods=1).mean())

    # df_weather_data_c.loc[
    #     df_weather_data_c["municipality"] == municipality, impute_cols
    # ] = temp_df[impute_cols]

   
df_weather_data_c.reset_index(inplace=True)

In [69]:
na_overview(df_weather_data_c)

(744600, 15)


,# missing values,% missing values
coco,48329,0.064906
prcp,110305,0.148140
snow,718320,0.964706
tsun,718320,0.964706


In [70]:
grouped_weather_1 = df_weather_data_c.groupby("municipality").apply(
    lambda x: x.isna().sum()
)
grouped_weather_1 = grouped_weather_1.query("(prcp < 8760) & (prcp > 0)")
grouped_weather_1

,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
municipality,,,,,,,,,,,,,,,
Aalborg,0,0,233,0,0,0,1,0,0,8760,0,8760,0,0,0
Aarhus,0,0,241,0,0,0,1,0,0,8760,0,8760,0,0,0
Billund,0,0,235,0,0,0,3495,0,0,8760,0,8760,0,0,0
Esbjerg,0,0,232,0,0,0,3495,0,0,8760,0,8760,0,0,0
Fanø,0,0,232,0,0,0,3495,0,0,8760,0,8760,0,0,0
Hedensted,0,0,235,0,0,0,3495,0,0,8760,0,8760,0,0,0
Herning,0,0,232,0,0,0,1,0,0,8760,0,8760,0,0,0
Holstebro,0,0,232,0,0,0,1,0,0,8760,0,8760,0,0,0
Ikast-Brande,0,0,232,0,0,0,1,0,0,8760,0,8760,0,0,0


After interpolating with `interpolate.("time")` there is still missing values for prcp. This happens to be the case due to our interpolation method which relies on continuity of the points. *Perhaps change interpolation method*

*Comment 2024-11-11: As you see above, the consistent number in `prcp` may be due to missing values up to some point. When looking at Billund this is the case. One can assume that is the case for the remaining municipalities.*


In [71]:
grouped_weather_1.query("prcp == 1").index

Index(['Aalborg', 'Aarhus', 'Herning', 'Holstebro', 'Ikast-Brande',
       'Norddjurs', 'Rebild', 'Silkeborg', 'Syddjurs', 'Viborg'],
      dtype='object', name='municipality')

In [81]:
# Making sure to impute the last remaining value as these are feasible to impute

for municipality in grouped_weather_1.query("prcp == 1").index:
    
    temp_df = df_weather_data_c[
    df_weather_data_c["municipality"] == municipality
    ]

    display(temp_df.head(3))

    # temp_df["prcp"] = temp_df["prcp"].fillna(temp_df["prcp"].rolling(
    #     window=3, min_periods=1).mean())

    temp_df["prcp"] = temp_df["prcp"].fillna(method='bfill')

    df_weather_data_c.loc[df_weather_data_c["municipality"] == municipality, 
                         "prcp"] = temp_df["prcp"]

,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
83,2021-12-31 23:00:00,835e7bc2-a1b6-11ef-bd47-b3d3ea7a5132,5.0,7375.263831,5.4,Aalborg,NaN,1010.5,100.0,NaN,5.4,NaN,250.0,25.9,18.4
165,2022-01-01 00:00:00,835e7bcc-a1b6-11ef-bd47-b3d3ea7a5132,5.0,7375.263831,6.1,Aalborg,0.0,1011.3,100.0,NaN,6.1,NaN,260.0,29.6,18.4
184,2022-01-01 01:00:00,835e7be0-a1b6-11ef-bd47-b3d3ea7a5132,5.0,7375.263831,6.4,Aalborg,0.0,1011.6,100.0,NaN,6.4,NaN,260.0,27.8,20.5


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
64,2021-12-31 23:00:00,835b65e0-a1b6-11ef-bd47-b3d3ea7a5132,5.0,30018.490633,7.9,Aarhus,NaN,1012.0,98.0,NaN,8.2,NaN,0.0,24.1,0.0
163,2022-01-01 00:00:00,835b65ea-a1b6-11ef-bd47-b3d3ea7a5132,4.0,30018.490633,8.1,Aarhus,0.0,1012.6,98.0,NaN,8.4,NaN,0.0,24.1,0.0
183,2022-01-01 01:00:00,835b65fe-a1b6-11ef-bd47-b3d3ea7a5132,5.0,30018.490633,8.1,Aarhus,0.0,1013.3,98.0,NaN,8.4,NaN,0.0,22.2,0.0


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
2,2021-12-31 23:00:00,835ffc72-a1b6-11ef-a409-035ec2b753b3,5.0,20042.564359,6.0,Herning,NaN,1012.4,100.0,NaN,6.0,NaN,230.0,20.4,14.8
91,2022-01-01 00:00:00,835ffc7c-a1b6-11ef-a409-035ec2b753b3,5.0,20042.564359,5.9,Herning,0.0,1012.8,100.0,NaN,5.9,NaN,240.0,20.4,14.8
186,2022-01-01 01:00:00,835ffc90-a1b6-11ef-a409-035ec2b753b3,5.0,20042.564359,6.0,Herning,0.0,1013.0,100.0,NaN,6.0,NaN,230.0,20.4,9.4


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
70,2021-12-31 23:00:00,8544d17a-a1b6-11ef-bd84-97f2ced146fd,5.0,11184.887627,6.0,Holstebro,NaN,1012.4,100.0,NaN,6.0,NaN,230.0,20.4,14.8
145,2022-01-01 00:00:00,8544d18e-a1b6-11ef-bd84-97f2ced146fd,5.0,11184.887627,5.9,Holstebro,0.0,1012.8,100.0,NaN,5.9,NaN,240.0,20.4,14.8
177,2022-01-01 01:00:00,8544d198-a1b6-11ef-bd84-97f2ced146fd,5.0,11184.887627,6.0,Holstebro,0.0,1013.0,100.0,NaN,6.0,NaN,230.0,20.4,9.4


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
71,2021-12-31 23:00:00,85438748-a1b6-11ef-9bf1-5b2646d1ae31,5.0,28749.383609,6.0,Ikast-Brande,NaN,1012.4,100.0,NaN,6.0,NaN,230.0,20.4,14.8
148,2022-01-01 00:00:00,85438752-a1b6-11ef-9bf1-5b2646d1ae31,5.0,28749.383609,5.9,Ikast-Brande,0.0,1012.8,100.0,NaN,5.9,NaN,240.0,20.4,14.8
204,2022-01-01 01:00:00,85438766-a1b6-11ef-9bf1-5b2646d1ae31,5.0,28749.383609,6.0,Ikast-Brande,0.0,1013.0,100.0,NaN,6.0,NaN,230.0,20.4,9.4


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
14,2021-12-31 23:00:00,8a4f36c4-a1b6-11ef-9bf1-5b2646d1ae31,5.0,16817.506854,7.9,Norddjurs,NaN,1012.0,98.0,NaN,8.2,NaN,0.0,24.1,0.0
86,2022-01-01 00:00:00,8a4f36ce-a1b6-11ef-9bf1-5b2646d1ae31,4.0,16817.506854,8.1,Norddjurs,0.0,1012.6,98.0,NaN,8.4,NaN,0.0,24.1,0.0
233,2022-01-01 01:00:00,8a4f36e2-a1b6-11ef-9bf1-5b2646d1ae31,5.0,16817.506854,8.1,Norddjurs,0.0,1013.3,98.0,NaN,8.4,NaN,0.0,22.2,0.0


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
0,2021-12-31 23:00:00,86951576-a1b6-11ef-913c-27e1cf788e95,5.0,20676.8573,5.4,Rebild,NaN,1010.5,100.0,NaN,5.4,NaN,250.0,25.9,18.4
131,2022-01-01 00:00:00,86951580-a1b6-11ef-913c-27e1cf788e95,5.0,20676.8573,6.1,Rebild,0.0,1011.3,100.0,NaN,6.1,NaN,260.0,29.6,18.4
171,2022-01-01 01:00:00,8695158a-a1b6-11ef-913c-27e1cf788e95,5.0,20676.8573,6.4,Rebild,0.0,1011.6,100.0,NaN,6.4,NaN,260.0,27.8,20.5


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
18,2021-12-31 23:00:00,835c9cbc-a1b6-11ef-a409-035ec2b753b3,5.0,30433.429704,6.0,Silkeborg,NaN,1012.4,100.0,NaN,6.0,NaN,230.0,20.4,14.8
154,2022-01-01 00:00:00,835c9cc6-a1b6-11ef-a409-035ec2b753b3,5.0,30433.429704,5.9,Silkeborg,0.0,1012.8,100.0,NaN,5.9,NaN,240.0,20.4,14.8
219,2022-01-01 01:00:00,835c9cda-a1b6-11ef-a409-035ec2b753b3,5.0,30433.429704,6.0,Silkeborg,0.0,1013.0,100.0,NaN,6.0,NaN,230.0,20.4,9.4


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
66,2021-12-31 23:00:00,8aa6ffa8-a1b6-11ef-a714-5bb2c7a11fbf,5.0,4267.646212,7.9,Syddjurs,NaN,1012.0,98.0,NaN,8.2,NaN,0.0,24.1,0.0
160,2022-01-01 00:00:00,8aa6ffb2-a1b6-11ef-a714-5bb2c7a11fbf,4.0,4267.646212,8.1,Syddjurs,0.0,1012.6,98.0,NaN,8.4,NaN,0.0,24.1,0.0
221,2022-01-01 01:00:00,8aa6ffbc-a1b6-11ef-a714-5bb2c7a11fbf,5.0,4267.646212,8.1,Syddjurs,0.0,1013.3,98.0,NaN,8.4,NaN,0.0,22.2,0.0


,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
68,2021-12-31 23:00:00,8c296a5a-a1b6-11ef-bd47-b3d3ea7a5132,5.0,18627.607612,6.8,Viborg,NaN,1011.6,100.0,NaN,6.8,NaN,249.0,20.4,13.0
89,2022-01-01 00:00:00,8c296a6e-a1b6-11ef-bd47-b3d3ea7a5132,5.0,18627.607612,6.8,Viborg,0.0,1012.4,100.0,NaN,6.8,NaN,250.0,22.2,13.0
226,2022-01-01 01:00:00,8c296a78-a1b6-11ef-bd47-b3d3ea7a5132,5.0,18627.607612,6.7,Viborg,0.0,1012.9,100.0,NaN,6.7,NaN,246.0,22.2,11.1


All the remaining missing values was luckily at the start of the series. By doing a NOCB we can ensure that we have no missing values left.

In [84]:
grouped_weather_2 = df_weather_data_c.groupby("municipality").apply(
    lambda x: x.isna().sum()
)
grouped_weather_2 = grouped_weather_2.query("(prcp < 8760) & (prcp > 0)")
grouped_weather_2

,hourutc,id,coco,distance_to_station,dwpt,municipality,prcp,pres,rhum,snow,temp,tsun,wdir,wpgt,wspd
municipality,,,,,,,,,,,,,,,
Billund,0,0,235,0,0,0,3495,0,0,8760,0,8760,0,0,0
Esbjerg,0,0,232,0,0,0,3495,0,0,8760,0,8760,0,0,0
Fanø,0,0,232,0,0,0,3495,0,0,8760,0,8760,0,0,0
Hedensted,0,0,235,0,0,0,3495,0,0,8760,0,8760,0,0,0
Kerteminde,0,0,246,0,0,0,3495,0,0,8760,0,8760,0,0,0
Middelfart,0,0,246,0,0,0,3495,0,0,8760,0,8760,0,0,0
Nordfyn,0,0,246,0,0,0,3495,0,0,8760,0,8760,0,0,0
Odense,0,0,246,0,0,0,3495,0,0,8760,0,8760,0,0,0
Varde,0,0,232,0,0,0,3495,0,0,8760,0,8760,0,0,0


In [85]:
na_overview(df_weather_data_c)

(744600, 15)


,# missing values,% missing values
coco,48329,0.064906
prcp,110295,0.148127
snow,718320,0.964706
tsun,718320,0.964706


In [86]:
# Write imputation to database

from pyspark.sql.functions import col

spark.createDataFrame(df_weather_data_c)\
    .withColumn("id", col("id").cast(StringType())).write \
    .format("org.apache.spark.sql.cassandra") \
    .mode('append') \
    .options(table="weather_data", keyspace=keyspace) \
    .save()

24/11/13 13:28:58 WARN TaskSetManager: Stage 4 contains a task of very large size (11259 KiB). The maximum recommended task size is 1000 KiB.


### Synchronization

In [ ]:
def sync_weather_to_gas(
    weather_df: pd.DataFrame,
    gas_df: pd.DataFrame,
    time_weather: str,
    time_gas: str,
    weather_property: str,
    municipality: str,
    accumulation_mtd: str = "mean",
):
    # Dropping "id" as it will be a hurdle when aggregating
    weather_df = weather_df.drop(labels="id", axis=1)
    gas_df = gas_df.drop(labels="_id", axis=1)

    # Filter weather_df w.r.t. municipality
    weather_df = weather_df.query(f"municipality == '{municipality}'")

    # Sort the DataFrames in temporal order
    weather_df = weather_df.sort_values(time_weather, ascending=True)
    gas_df = gas_df.sort_values(time_gas, ascending=True)

    # Rename the time_gas column to time_weather
    gas_df = gas_df.rename(columns={time_gas: time_weather})

    # Set the index to their time column
    weather_df.set_index(time_weather, inplace=True)
    gas_df.set_index(time_weather, inplace=True)

    # Only have mean implemented, but could be further extended
    if accumulation_mtd == "mean":
        weather_df = weather_df.groupby("municipality").resample("D").mean()

    # Merging the two DataFrames on their respective time column
    combined_df = pd.merge(weather_df, gas_df, on=time_weather)

    return combined_df[[weather_property, "PurchasePriceDKK_kWh"]]

In [ ]:
random.seed(1234341)  # Setting the seed for reproducability

# Choose a random municipality
rand_municipality = random.choice(df_weather_data_c["municipality"].unique())
df_random = sync_weather_to_gas(
    df_weather_data,
    df_gas_prices,
    "hourutc",
    "GasDay",
    "temp",
    rand_municipality,
)

fig, ax1 = plt.subplots(figsize=(10, 5))

# Plotting the temperatures
ax1.plot(df_random.index, df_random["temp"], color="b", label="Temperature")
ax1.set_xlabel("time")
ax1.set_ylabel("Temperature", color="b")
ax1.tick_params(axis="y", labelcolor="b")

# Plotting the gas purchase prices
ax2 = ax1.twinx()
ax2.plot(
    df_random.index,
    df_random["PurchasePriceDKK_kWh"],
    color="r",
    label="Gas purchase price",
)
ax2.set_ylabel("PurchasePriceDKK_kWh", color="r")
ax2.tick_params(axis="y", labelcolor="r")

fig.legend(loc="upper right")
plt.title(f"Temperature in {rand_municipality} and gas purchase prices")

plt.show()